In [1]:
import tensorflow as tf
import numpy as np

In [43]:
tf.reset_default_graph()

sentences = ["i love you", "he loves me", "she likes baseball", "i hate you", "sorry for that", "this is awful"]
labels = [1,1,1,0,0,0]
#paramater
embedding_size = 2
n_hidden = 5
n_step = 3
n_class = 2

word_list = ' '.join(sentences).split()
word_list = list(set(word_list))
word_num = {w:i for i,w in enumerate(word_list)}
num_word = {i:w for i,w in enumerate(word_list)}
vocab_size = len(word_list)

input_batch = []
target_batch = []
for sen in sentences:
    input_batch.append([word_num[n] for n in sen.split()])

for out in labels:
    target_batch.append(np.eye(n_class)[out])
    
X = tf.placeholder(tf.int32,[None,n_step])
Y = tf.placeholder(tf.int32,[None,n_class])
out = tf.Variable(tf.random_normal([n_hidden*2,n_class]))

embedding = tf.Variable(tf.random_uniform([vocab_size,embedding_size]))
cell_in = tf.nn.embedding_lookup(embedding,X)

#model
fw_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden)
bw_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden)

hidden_outputs,final_state = tf.nn.bidirectional_dynamic_rnn(fw_cell,bw_cell,cell_in,dtype=tf.float32)
#hidden_outputs[2,batch_size,n_step,hidden_size]
#final_state [2,2,batch_size,hidden_size]
output = tf.concat([hidden_outputs[0],hidden_outputs[1]],2)#6,3,10
final_state = tf.concat([final_state[1][0],final_state[1][1]],1)#6,10
final_state = tf.expand_dims(final_state,2)
attn_weight = tf.squeeze(tf.matmul(output,final_state),2)#6,3
soft_attn_weights = tf.nn.softmax(attn_weight, 1)#6,3
context = tf.matmul(tf.transpose(output,[0,2,1]),tf.expand_dims(soft_attn_weights,2))#6,10,1
context = tf.squeeze(context,2)#6,10
model = tf.matmul(context,out)#batch n_class

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model,labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

predict = tf.argmax(tf.nn.softmax(model),1)


sess = tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(5000):
    _,loss = sess.run([optimizer,cost],feed_dict={X:input_batch,Y:target_batch})
    
    if (i+1) %1000==0:
        print('epoch: %d, cost:%.6f' % ((i+1),loss))
    
test = 'sorry hate you'
tests = [np.asarray([word_num[n] for n in test.split()])]
pre = sess.run(predict,feed_dict={X:input_batch,Y:target_batch})
print(pre[0])
              


epoch: 1000, cost:0.002649
epoch: 2000, cost:0.000670
epoch: 3000, cost:0.000276
epoch: 4000, cost:0.000134
epoch: 5000, cost:0.000070
1


In [42]:
len(pre)

6